In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from lib.actor.LeeNet import LeeNetActor
from lib.models.backbone.plscore_RMT_mix import PLScoreRMT
from lib.models.LeeNet.score_RMT_center import ScorePureRMTCENTER
from lib.models.head.center_predictor import CenterPredictor
from lib.trainer.LeeNet_trainer import LeeNetTrainer
from lib.utils.base_funtion import build_dataloaders, get_optimizer_scheduler
from lib.config.cfg_loader import env_setting
from torch.nn.functional import l1_loss
from torch.nn import BCEWithLogitsLoss
from lib.utils.box_ops import giou_loss
from lib.utils.focal_loss import FocalLoss
import torch


def build_model(cfg):
    backbone = PLScoreRMT(cfg=cfg)
    
    stride = cfg.model.backbone.stride
    feat_sz = int(cfg.data.search.size / stride)
    head = CenterPredictor(inplanes=cfg.model.pureRMT.embed_dim[-1], channel=cfg.model.head.num_channels, feat_sz=feat_sz, stride=stride)
    model = ScorePureRMTCENTER(backbone, head, cfg)
    return model


cfg = env_setting(cfg_name='plscore_RMT_mix_center00001.yaml')
loader_train, loader_val = build_dataloaders(cfg)
data = None
# for i, _data in enumerate(loader_train, 1):
#     data = _data
#     break

In [ ]:
# net = build_model(cfg)
# 
# focal_loss = FocalLoss()
# objective = {'giou': giou_loss, 'l1': l1_loss, 'focal': focal_loss, 'cls': BCEWithLogitsLoss()}
# loss_weight = {'giou': cfg.train.GIOU_weight, 'l1': cfg.train.L1_weight, 'focal': 1., 'cls': 1.0}
# actor = LeeNetActor(net=net, objective=objective, loss_weight=loss_weight, cfg=cfg)

In [ ]:
# data = data.to(cfg.train.device)
# actor(data)

In [ ]:
checkpoint_path = "/media/star/data/Leezed/workspace/LeeNet/checkpoints/LeeNet_plscore_RMT_CENTER_no_down/ScorePureRMTCENTER_ep0320.pth.tar"
checkpoint_dict = torch.load(checkpoint_path,map_location={"cuda:3":"cuda:3"})['net']
print(checkpoint_dict.keys())

net = build_model(cfg)

model_dict = net.state_dict()
print(model_dict.keys())
state_dict = {k: v for k, v in checkpoint_dict.items() if k in model_dict.keys()}
print(state_dict.keys())
model_dict.update(state_dict)
net.load_state_dict(model_dict,strict=False)

In [ ]:


focal_loss = FocalLoss()
objective = {'giou': giou_loss, 'l1': l1_loss, 'focal': focal_loss, 'cls': BCEWithLogitsLoss()}
loss_weight = {'giou': cfg.train.GIOU_weight, 'l1': cfg.train.L1_weight, 'focal': 1., 'cls': 1.0}
actor = LeeNetActor(net=net, objective=objective, loss_weight=loss_weight, cfg=cfg)

optimizer, lr_scheduler = get_optimizer_scheduler(net, cfg)

trainer = LeeNetTrainer(actor=actor, loaders=[loader_train, loader_val], optimizer=optimizer, lr_scheduler=lr_scheduler, cfg=cfg)

trainer.train(cfg.train.epoch)